<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.18.5
Pandas:  1.0.5


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [16]:
# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 0)
print(bikes.head())



FileNotFoundError: [Errno 2] File dat/bikeshare.csv does not exist: 'dat/bikeshare.csv'

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [3]:
#ANSWER:
bikes = pd.read_csv('bikeshare.csv',header = 0)
bikes.head()


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [ ]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [ ]:
df = pd.read_excel('Iris.xls', sheet_name = 'Data')
df

So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [ ]:
#ANSWER
df = pd.read_excel('Iris.xls', sheet_name = 'Data', usecols = [0, 1, 2, 3, 4])
df

### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [ ]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [ ]:
##url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
df = pd.read_html(url)
df

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [ ]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [ ]:
#ANSWER
bikes.shape



What are the column names in `bikes`?

In [ ]:
#ANSWER
bikes.columns


What are the data types of these columns?

In [11]:
#ANSWER
bikes.dtypes


NameError: name 'bikes' is not defined

In [ ]:
bikes.info()

What is the (row) index for this DataFrame?

In [ ]:
#ANSWER
bikes.index



https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [ ]:
#ANSWER
bikes['season']

In [ ]:
print(bikes['atemp'].head())
print(type(bikes['atemp']))
print(bikes[['atemp']].head())
print(type(bikes[['atemp']]))


In [ ]:
print(bikes.atemp[:4])

In [ ]:
bikes[['season']]

How would we use object notation to show the first 4 rows of `atemp`?

In [ ]:
#ANSWER
bikes[['atemp']].head(n=4)


Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [ ]:
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

What is the correct way to access the 1st row of the DataFrame by its index?

In [ ]:
#ANSWER
bikes[0:1]


What is the correct way to access the 2nd column of the DataFrame by its index?

In [ ]:
#ANSWER
bikes[1:2]


In [ ]:
bikes.iloc[:, 1]
help(bikes.loc)


## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [ ]:
bikes.isnull().head()

However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [ ]:
#ANSWER
empty = bikes.isnull()
bikes[empty == True].count()

In [ ]:
bikes.isnull().sum()


If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [ ]:
#ANSWER
bikes[bikes.isnull() == True].count()

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [ ]:

#ANSWER

def halver(value):
    if value == 0 :
      print("Unable to divide by zero")
    else :
      print("half of this number is: " + str(value/2))
    return 

halver(1001)

halver(999)

halver(0)

In [ ]:
def divide(dividend, divisor) :
    if divisor == 0 :
        quotient = np.nan
    else:
        quotient = dividend / divisor
    return (quotient)

print(divide ( 3,0))
divide ( 3,2)

Apply the Pandas `isna` function to the following data objects:

In [ ]:
x = 2.3
y = np.nan
print(x, y)

In [ ]:
#ANSWER
print(pd.isna(x))
print(pd.isna(y))

In [ ]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)
print(pd.isna(array))

In [ ]:
#ANSWER


How is the pandas I/O parameter `na_values` used?

? ANSWER: 

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [ ]:
#ANSWER
bikes['season'].value_counts()

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [ ]:
#ANSWER
print(bikes[['instant','dteday','windspeed']])


Compute and print the overall minimum and maximum of the numeric data columns:

In [ ]:
##bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
##bikes_min, bikes_max

bikes[['instant','dteday','windspeed']].min()
bikes[['instant','dteday','windspeed']].max()


### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [12]:
bikes['atemp'].quantile(0.5)

NameError: name 'bikes' is not defined

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [ ]:
#ANSWER
bikes['atemp'].quantile((0.1,0.25,0.5,0.75,0.9))

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [ ]:
type(bikes['atemp'])

In [ ]:
bikes.sample(5)

Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [4]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [ ]:
#ANSWER


Here is a random sample of `atemp_level`:

In [5]:
atemp_level.sample(5)          

11034    (0.25, 0.5]
3742     (0.5, 0.75]
10771    (0.25, 0.5]
15580    (0.25, 0.5]
9625     (0.25, 0.5]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [6]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

3280     warm
2711     warm
2817     mild
11358    mild
376      cool
Name: atemp, dtype: category
Categories (4, object): [cool < mild < warm < hot]

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [ ]:
#ANSWER
bikes['atemp_level'] = atemp_level
bikes.head()

In [ ]:
bikes[(bikes.atemp_level == 'mild')&(bikes.season == 2)]


*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [18]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [ ]:
# Cleaning Data

## Load Data

Load rock.csv and clean the dataset.

In [21]:
rock = pd.read_csv('../DATA/rock.csv',header = 0)


,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,NaN,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


## Check Column Names

Check column names and clean.

In [100]:
rock.columns

#create new dataframe to remove columns that are aggregated from other columns
rock2 = rock.drop(columns = ['COMBINED','F*G'])
rock2.head()

,Song Clean,ARTIST CLEAN,Release Year,First?,Year?,PlayCount
0,Caught Up in You,.38 Special,1982,1,1,82
1,Fantasy Girl,.38 Special,NaN,1,0,3
2,Hold On Loosely,.38 Special,1981,1,1,85
3,Rockin' Into the Night,.38 Special,1980,1,1,18
4,Art For Arts Sake,10cc,1975,1,1,1


# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [101]:
#find out which columns contain null values, and how many
print(rock2.isnull().sum())

#replace this null values with 0 in a new dataframe
rock3 = rock2.fillna(0)

#check that the function worked correctly
rock3.isnull().sum()


Song Clean        0
ARTIST CLEAN      0
Release Year    577
First?            0
Year?             0
PlayCount         0
dtype: int64


Song Clean      0
ARTIST CLEAN    0
Release Year    0
First?          0
Year?           0
PlayCount       0
dtype: int64

In [102]:
rock3

,Song Clean,ARTIST CLEAN,Release Year,First?,Year?,PlayCount
0,Caught Up in You,.38 Special,1982,1,1,82
1,Fantasy Girl,.38 Special,0,1,0,3
2,Hold On Loosely,.38 Special,1981,1,1,85
3,Rockin' Into the Night,.38 Special,1980,1,1,18
4,Art For Arts Sake,10cc,1975,1,1,1
...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,0,1,0,1
2226,Tube Snake Boogie,ZZ Top,1981,1,1,32
2227,Tush,ZZ Top,1975,1,1,109
2228,TV Dinners,ZZ Top,1983,1,1,1


# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 


In [135]:
#Replace error value in the release_year column
rock3 = rock3.replace({'SONGFACTS.COM' : 0})
rock3

#Convert column datatype to int
rock3 = rock3.astype({'Release Year' : int})
rock3.dtypes


Song Clean      object
ARTIST CLEAN    object
Release Year     int32
First?           int64
Year?            int64
PlayCount        int64
dtype: object

In [136]:
rock3.isnull().sum()

Song Clean      0
ARTIST CLEAN    0
Release Year    0
First?          0
Year?           0
PlayCount       0
dtype: int64

# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [137]:
rock3.max()

Song Clean      Ziggy Stardust
ARTIST CLEAN              a-ha
Release Year              2014
First?                       1
Year?                        1
PlayCount                  142
dtype: object

In [138]:
rock3.min()

Song Clean         #9 Dream
ARTIST CLEAN    .38 Special
Release Year              0
First?                    1
Year?                     0
PlayCount                 0
dtype: object

# Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [216]:
def ChooseSong(n):
    if rock3.iloc[n+1]['Release Year'] == 0 :
      print("Song Title : "+ str(rock3.iloc[n+1]['Song Clean']) + 
            "\nArtist Name : "+ str(rock3.iloc[n+1]['ARTIST CLEAN']) +
            "\nRelease Year could not be determined")
    elif rock3.iloc[n+1]['Release Year'] < 1970 :
      print("Song Title : "+ str(rock3.iloc[n+1]['Song Clean']) + 
            "\nArtist Name : "+ str(rock3.iloc[n+1]['ARTIST CLEAN']) +
            "\nRelease Date is before 1970")
    else :
     print("Song Title : "+ str(rock3.iloc[n+1]['Song Clean']) + 
            "\nArtist Name : "+ str(rock3.iloc[n+1]['ARTIST CLEAN']) +
            "\nRelease Date is after 1970")
    
    return 



ChooseSong(30)

ChooseSong(31)

ChooseSong(147)

Song Title : Sin City
Artist Name : AC/DC
Release Year could not be determined
Song Title : T.N.T.
Artist Name : AC/DC
Release Date is after 1970
Song Title : Rock Around the Clock
Artist Name : Bill Haley
Release Date is before 1970


## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [224]:
#create duplicate dataframe before conversion
rock4 = rock3
rock4

#function
rockcols = rock4.columns
rockcols

Index(['Song Clean', 'ARTIST CLEAN', 'Release Year', 'First?', 'Year?',
       'PlayCount'],
      dtype='object')

## Apply these functions to your dataset

## Describe the new float-only DataFrame.



> 
>
>




>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---



